In [1]:
from openai import OpenAI
from dotenv import load_dotenv
import json
import csv
import os

json_filename = 'prompts.json' # relative
csv_path = os.path.normpath('C:/Users/arwin/Documents/dev/APS360-PROJECT/data') 

load_dotenv()
client = OpenAI()


System Prompt

In [2]:
with open(json_filename, 'r') as json_file:
    data = json.load(json_file)

system_prompt = data['Billboard System Prompt']
print(system_prompt)



You are a helpful data annotator who only outputs JSON.
You will be given a list, where each entry has 3 elements: artist, song name, and genre label. The artist and song name will be filled out for you, but the genre label will be blank. Your job is to use your knowledge, internet, and all other resources to fill in the genre label. The genre label you enter should be one of:
"pop", "hiphop", "rock", "country", "jazz", "disco", "reggae", "metal", "classical", "blues"
Example:
input = [["Kanye West", "Stronger", ""],["Taylor Swift", "Cruel Summer", ""]]
output = [["Kanye West", "Stronger", "hiphop"],["Taylor Swift", "Cruel Summer", "pop"]]


User Prompt

In [3]:
# load csv

csv_data = []
filename = 'billboard_genres.csv'

with open(os.path.join(csv_path, filename), 'r', newline='') as csvfile:
    csv_reader = csv.reader(csvfile)
    for row in csv_reader:
        csv_data.append(row)

# Now csv data is a giant list ready to serialize for user prompts


'[["Drake Featuring Lil Durk", "Laugh Now Cry Later", ""], ["Yo Gotti Featuring E-40", "Law", ""], ["Young Thug Featuring Future", "Sup Mate", ""], ["Eminem Featuring Jessie Reyez", "Good Guy", ""], ["Queen", "Bohemian Rhapsody", ""], ["Metro Boomin Featuring Travis Scott, Kodak Black & 21 Savage", "No More", ""], ["Eminem", "Darkness", ""], ["ScHoolboy Q Featuring Kanye West", "THat Part", ""], ["Chuck Berry", "Run Rudolph Run", ""], ["Juice WRLD", "Armed And Dangerous", ""], ["Wham! Featuring George Michael", "Careless Whisper", ""], ["Meghan Trainor", "Me Too", ""], ["Calvin Harris Featuring Pharrell Williams, Katy Perry & Big Sean", "Feels", ""], ["Migos Featuring Travis Scott", "Kelly Price", ""], ["Logic Featuring Wiz Khalifa", "Indica Badu", ""], ["Brett Young", "Here Tonight", ""], ["Selena Gomez", "Kill Em With Kindness", ""], ["David Bowie", "Lazarus", ""], ["twenty one pilots", "Cancer", ""], ["NF", "When I Grow Up", ""], ["J Balvin", "Ginza", ""], ["FLETCHER", "Undrunk", ""

Main Loop

In [5]:
n = 50

csvfile = open('billboard_genres_filled.csv', 'w', newline='')
csv_writer = csv.writer(csvfile)
csv_writer.writerow(['Artist', 'Song', 'Genre'])

for i in range(1, len(csv_data), n):
    # prepare user prompt (get n rows from table)
    json_data = []
    for row in csv_data[i: i + n]:    
        json_data.append([cell for cell in row] + [""])

    user_prompt = json.dumps(json_data)

    # send prompt
    response = client.chat.completions.create(
        model="gpt-4-0125-preview",
        response_format={ "type": "json_object" },
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )

    # parse prompt
    response_dict = json.loads(response.choices[0].message.content)
    assert(len(response_dict)==1)
    for k,v in response_dict.items():
        genre_rows = v
        break
        
    # write to output csv
    for row in genre_rows:
        csv_writer.writerow(row)
    print(genre_rows)


# user_prompt = json.dumps(json_data)
# user_prompt

[['Drake Featuring Lil Durk', 'Laugh Now Cry Later', 'hiphop'], ['Yo Gotti Featuring E-40', 'Law', 'hiphop'], ['Young Thug Featuring Future', 'Sup Mate', 'hiphop'], ['Eminem Featuring Jessie Reyez', 'Good Guy', 'hiphop'], ['Queen', 'Bohemian Rhapsody', 'rock'], ['Metro Boomin Featuring Travis Scott, Kodak Black & 21 Savage', 'No More', 'hiphop'], ['Eminem', 'Darkness', 'hiphop'], ['ScHoolboy Q Featuring Kanye West', 'THat Part', 'hiphop'], ['Chuck Berry', 'Run Rudolph Run', 'rock'], ['Juice WRLD', 'Armed And Dangerous', 'hiphop'], ['Wham! Featuring George Michael', 'Careless Whisper', 'pop'], ['Meghan Trainor', 'Me Too', 'pop'], ['Calvin Harris Featuring Pharrell Williams, Katy Perry & Big Sean', 'Feels', 'pop'], ['Migos Featuring Travis Scott', 'Kelly Price', 'hiphop'], ['Logic Featuring Wiz Khalifa', 'Indica Badu', 'hiphop'], ['Brett Young', 'Here Tonight', 'country'], ['Selena Gomez', 'Kill Em With Kindness', 'pop'], ['David Bowie', 'Lazarus', 'rock'], ['twenty one pilots', 'Cancer'